# Quantum Evolution Strategy on Feasible Solution Space

In [1]:
import numpy as np 
import math

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit import Aer
from qiskit.primitives import Sampler, BackendSampler
from qiskit.circuit.library import TwoLocal

from src.QuantumEvolutionStrategy import QuantumEvolutionStrategy

## 1. Importing a Model
In this notebook, we'll begin by bringing in a model.

In [2]:
target_mdl = QuadraticProgram()
target_mdl.read_from_lp_file('instance/seq2.lp')

N = target_mdl.get_num_vars()
print(N)

print(target_mdl.prettyprint())

22
Problem name: docplex_model1

Minimize
  -20*x_11_6_14_7_13*x_13_7_13_8_12 - 20*x_12_6_22_7_21*x_16_7_21_8_20
  - 20*x_16_7_21_8_20*x_17_8_20_9_19 - 20*x_18_9_20_10_19*x_19_10_19_11_18
  - 20*x_1_1_10_2_9*x_4_2_9_3_8 - 20*x_5_2_10_3_9*x_6_3_9_4_8 - 0.5*x_0_1_9_2_8
  - 2.1*x_10_5_22_6_21 - 0.6*x_11_6_14_7_13 - 0.6*x_12_6_22_7_21
  - 2.1*x_13_7_13_8_12 - x_14_7_16_8_15 - 1.4*x_15_7_18_8_17 - x_16_7_21_8_20
  - 0.5*x_17_8_20_9_19 - 0.5*x_18_9_20_10_19 - 2.5*x_19_10_19_11_18
  - 0.5*x_1_1_10_2_9 - 0.6*x_20_13_20_14_19 - 1.3*x_21_15_21_16_20
  - 0.6*x_2_1_14_2_13 - 0.6*x_3_1_22_2_21 - 1.5*x_4_2_9_3_8 - 1.5*x_5_2_10_3_9
  - 3.3*x_6_3_9_4_8 - 3.3*x_7_3_10_4_9 - 3.3*x_8_4_10_5_9 - 2.1*x_9_5_14_6_13

Subject to
  Linear constraints (198)
    x_0_1_9_2_8 + x_1_1_10_2_9 <= 1  'c1'
    x_0_1_9_2_8 + x_2_1_14_2_13 <= 1  'c2'
    x_0_1_9_2_8 + x_3_1_22_2_21 <= 1  'c3'
    x_0_1_9_2_8 + x_4_2_9_3_8 <= 1  'c4'
    x_0_1_9_2_8 + x_5_2_10_3_9 <= 1  'c5'
    x_0_1_9_2_8 + x_6_3_9_4_8 <= 1  'c6'
    x_

## 2. Solving with MIP Solvers
We optionally solve the problem using a MIP solver.

In [3]:
cplex_result = CplexOptimizer().solve(target_mdl)
cplex_optval = cplex_result.fval
cplex_xsol = cplex_result.x.astype(int)

print(
        '-'*110,
        '[CPLEX Result]',
        f'   optimal value: {cplex_optval:.3f}',
        f'   optimal solution (binary): {cplex_xsol}',
        f'   status: {cplex_result.status.name}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -42.100
   optimal solution (binary): [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
   status: SUCCESS
--------------------------------------------------------------------------------------------------------------


## 3. Constructing a Feasible Solution Space
- We have set up a logic system to identify feasible solutions that meet the one-hot inequality constraints. 
- In this notebook, however, we go through the process of listing and enumerating these solutions in order to validate the performance of the optimizer.

Note: In our future work, we plan to enhance the efficiency of our approach by introducing an encoding method that utilizes a tree structure. This will allow us to replace the exhaustive enumeration process with a more efficient tree-based search method.

In [4]:
l_const_info = target_mdl.get_linear_constraint(0)
a = l_const_info.linear.to_array()
b = l_const_info.rhs

A = []
for j in range(0, target_mdl.get_num_linear_constraints()):
    l_const_info = target_mdl.get_linear_constraint(j)
    a = l_const_info.linear.to_array().astype(int)
    b = l_const_info.rhs
    A.append(a)

A = np.asarray(A, dtype=int)

In [5]:
labels = [[]]
stock = []
l_zeros = np.arange(A.shape[1])
stride = [l_zeros]

N = A.shape[1]

def search(A):
    global l_zeros, labels, stock, count, N

    for k in range(len(stride[-1])):
        idx = stride[-1][k]
        stock.append(idx)
        labels.append(stock.copy())

        v = np.zeros(N, dtype=int)
        for i in range(A.shape[0]):
            r = A[i]
            if r[idx] == 1:
                v += r
        
        y = np.argwhere(v == 0).flatten()
        l_zeros = np.array([i for i in y if i in stride[-1] and i > idx])

        if len(l_zeros) > 0:
            stride.append(l_zeros)
            search(A)
        else:
            stock.pop()
            
        if k == len(stride[-1]) - 1:
            stride.pop()
            if len(stock) == 0: continue
            stock.pop()
            break
    
    return labels

Sol = search(A)

## 4. Implementing Efficient Encoding
We optimize our solution generation process. Please refer to the notes in 3 for details.

In [6]:
def convert_bit_sol(x):
    y = [str(i) for i in x]
    y = ''.join(y)
    idx = int(y, 2) % len(Sol)

    v = np.zeros(N, dtype=int)
    v[Sol[idx]] = 1

    return v

## 5. Preparing for Quantum Computation
We now get ready for quantum computing.

In [7]:
num_qubits = math.ceil(math.log2(len(Sol)))

print(f'number of system qubits: {num_qubits}')

number of system qubits: 6


In [8]:
reps = 2

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    entanglement = 'pairwise', 
    reps = reps,
    insert_barriers = True
    )

n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

number of params: 18


In [9]:
shots = 2**4
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 16


## 6. Setting Up the Quantum Algorithm
We configure the quantum algorithm for our problem.

In [10]:
Q_obj = target_mdl.objective.quadratic.coefficients.copy()
l_diag = target_mdl.objective.linear
for k in range(N):
    Q_obj[k,k] += l_diag[k]
Q_obj = Q_obj.toarray()

Q_normalize = np.copy(Q_obj)
s = np.ones(N, dtype=int)
s_noralize = -s @ Q_obj @ s
Q_normalize /= s_noralize

def objective(x):
    val = x @ Q_normalize @ x
    return val

def indicator(x):
    val = x @ Q_obj @ x
    return val

def equation(x):
    return 0

## 7. Running the Algorithm and Checking the Solution
Finally, we execute the algorithm and verify the solution obtained.

In [11]:
QuantumSolver = QuantumEvolutionStrategy()
QuantumSolver.set_optimizer(N, ansatz, sampler)

QuantumSolver.run(
    objective = objective, 
    equation = equation, 
    indicator = indicator, 
    convert_bit_sol = convert_bit_sol,
    maxiter = 100, 
    refval = cplex_optval
    )

[I 2023-12-08 12:39:58,421] A new study created in memory with name: Quantum Optimization


step = 0 	 objval = -4.600
step = 1 	 objval = -42.100


In [12]:
x = QuantumSolver.x
objval = x @ Q_obj @ x

print(
        '-'*110,
        '[Quantum Result]',
        f'   objective:   {objval:.3f}',
        f'   solution:    {x}',
        f'   is feasible: {target_mdl.is_feasible(x)}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[Quantum Result]
   objective:   -42.100
   solution:    [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
   is feasible: True
--------------------------------------------------------------------------------------------------------------
